<a href="https://colab.research.google.com/github/tuky-g/LLM-QAproject/blob/main/2_QA_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [1]:
!pip install jupyter_bokeh
!pip install langchain_community
!pip install -qU chromadb langchain-chroma
!pip install tiktoken
!pip install --upgrade --quiet  langchain sentence_transformers
!pip install langchain_huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.

In [2]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
# from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from chromadb.config import Settings

In [4]:
import panel as pn
import param

In [5]:
import os
import sys
sys.path.append('../..')
import panel as pn
import param

# import panel as pn  # GUI
# pn.extension()


In [6]:
HF_API_KEY = os.environ.get('HF_TOKEN')

# conect to drive

In [7]:
#connect to folder in drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# retrival

In [19]:
from google.colab import output
output.enable_custom_widget_manager()

In [8]:
persist_directory='/content/drive/MyDrive/LLM-QAproject/docs/chroma4/'
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"
                                  # ,clean_up_tokenization_spaces=True  # Add this line
                                  )


<ipython-input-8-29f6db5fb9bf>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# load from disk
vectordb = Chroma(
    collection_name="foo",
    persist_directory=persist_directory,
    embedding_function=embeddings
)

vectordb._collection


<ipython-input-9-fba668ad6dc4>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


Collection(name=foo)

In [20]:
print(vectordb._collection.count())

886


In [40]:
question = "What is a NetBEUI?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

# chat

In [41]:
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


# Initialize your retriever
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4}) # Your retriever

# Initialize the language model
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

# Define the prompt to contextualize the question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Define the prompt to answer the question
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise and finish the sentences."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create the retrieval chain
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

In [42]:
# Usage example
chat_history = []  # Collect chat history here (a sequence of messages)
query = "What is a NetBEUI?"
response = rag_chain.invoke({"input": query, "chat_history": chat_history})
print(response)

{'input': 'What is a NetBEUI?', 'chat_history': [], 'context': [Document(metadata={'page': 4, 'source': '/content/drive/MyDrive/LLM-QAproject/pdf_vlite/Copy of 01. Advanced Penetration Testing.pdf'}, page_content='Figure 1.9 Blank document carrying macro payload.\nFigure 1.10 A little more convincing.\nFigure 1.11 Initial basic Command and Control infrastructure.\nFigure 1.12 The completed attack with complete access to the medical\nrecords.\nChapter 2: Stealing Research\nFigure 2.1 Permit all local Java code to run in the browser.\nFigure 2.2 Java applet running in the browser.\nFigure 2.3 The upgraded framework handles multiple hosts and\noperating systems.\nChapter 3: Twenty-First Century Heist\nFigure 3.1 The beauty of this setup is that if your C2 is disrupted by\nsecurity operations, you can point your DNS at another server.\nFigure 3.2 A basic intrusion monitoring setup.\nFigure 3.3 Mmmmmm. Stealthy.\nChapter 4: Pharma Karma\nFigure 4.1 This image from cvedetails shows 56 code e

In [55]:
response.keys()


dict_keys(['input', 'chat_history', 'context', 'answer'])

# chat

In [45]:


def load_db(k):
    persist_directory='/content/drive/MyDrive/LLM-QAproject/docs/chroma4/'
    # define embedding
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"
                                  # ,clean_up_tokenization_spaces=True  # Add this line
                                  )
    # load db
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
    # define retriever
    retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    # model
    llm = HuggingFaceEndpoint(
        repo_id="HuggingFaceH4/zephyr-7b-beta",
        task="text-generation",
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    )
    chat_model = ChatHuggingFace(llm=llm)
    #memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    # qa
    qa = ConversationalRetrievalChain.from_llm(
        chat_model,
        retriever=retriever,
        memory=memory
    )
    return qa

In [49]:
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

def load_db2(k):

  # Initialize your retriever
  retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4}) # Your retriever

  # Initialize the language model
  llm = HuggingFaceEndpoint(
      repo_id="HuggingFaceH4/zephyr-7b-beta",
      task="text-generation",
      max_new_tokens=512,
      do_sample=False,
      repetition_penalty=1.03,
  )

  # Define the prompt to contextualize the question
  contextualize_q_system_prompt = (
      "Given a chat history and the latest user question "
      "which might reference context in the chat history, "
      "formulate a standalone question which can be understood "
      "without the chat history. Do NOT answer the question, just "
      "reformulate it if needed and otherwise return it as is."
  )
  contextualize_q_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", contextualize_q_system_prompt),
          MessagesPlaceholder("chat_history"),
          ("human", "{input}"),
      ]
  )


  history_aware_retriever = create_history_aware_retriever(
      llm, retriever, contextualize_q_prompt
  )

  # Define the prompt to answer the question
  qa_system_prompt = (
      "You are an assistant for question-answering tasks. Use "
      "the following pieces of retrieved context to answer the "
      "question. If you don't know the answer, just say that you "
      "don't know. Use three sentences maximum and keep the answer "
      "concise and finish the sentences."
      "\n\n"
      "{context}"
  )
  qa_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", qa_system_prompt),
          MessagesPlaceholder("chat_history"),
          ("human", "{input}"),
      ]
  )

  # Create the question-answering chain
  question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

  # Create the retrieval chain
  rag_chain = create_retrieval_chain(
      history_aware_retriever, question_answer_chain
  )
  return rag_chain

In [46]:


# class cbfs(param.Parameterized):
#     chat_history = param.List([])
#     answer = param.String("")
#     db_query  = param.String("")

#     def __init__(self,  **params):
#         super(cbfs, self).__init__( **params)
#         self.panels = []
#         self.qa = load_db(4)


#     def convchain(self, query):
#         if not query:
#             return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
#         result = self.qa({"question": query, "chat_history": self.chat_history})
#         self.chat_history.extend([(query, result["answer"])])
#         self.db_query = result["question"]
#         self.answer = result['answer']
#         self.panels.extend([
#             pn.Row('User:', pn.pane.Markdown(query, width=600)),
#             pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600))
#         ])
#         inp.value = ''  #clears loading indicator when cleared
#         return pn.WidgetBox(*self.panels,scroll=True)


#     def clr_history(self,count=0):
#         self.chat_history = []
#         return


In [61]:


class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.rag_chain = load_db2(4)


    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.rag_chain.invoke({"input": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["input"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return

In [62]:
import panel as pn  # GUI
pn.extension()  # Load Panel extension

In [63]:
cb = cbfs()


inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)


tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()